# ASHE Table 7

-----

### Requirements

"Annual Summary of Hours and Earnings"


#### Observations & Dimensions

The `observations` are the numbers in the percentile columns.

The required dimensions are:

* **Geography** - in the `Code` column, one letter followed by 8 digits
* **Percentiles** - 10,20,30, etc
* **Time** - year, 4 digits
* **Gender** - Male, Female, All
* **Working Pattern** - Full time, Part time, All
* **Statistics** - The "topic" of the dataset, i.e "monthly pay net etc", in the filename

-----
    
Notes:

The "statistics" seems pointless because we're looking at one file. In production there are 24 per year per ASHE table.

It's always worth getting the file out of /sources and having a look over.

In [1]:
# [For Adnrew]


from databaker.framework import * # the gssutils module makes this databaker module redundant but need ons dev container
# from gssutils import *

# put your input-output files here
inputfile = "./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls"
outputfile = "pay_results.csv"
previewfile = "pay_results.html"
tidy_sheets = [] # for collating all the conversion segments turned dataframes

# load tabs
tabs = loadxlstabs(inputfile)


Loading file ./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls which has size 846336 bytes
Table names: ['Notes', 'All', 'Male', 'Female', 'Full-Time', 'Part-Time', 'Male Full-Time', 'Male Part-Time', 'Female Full-Time', 'Female Part-Time']


In [2]:
#show list of table names to choose from
# for tab in tabs:
   # print(tab.name)


# Filter out the tabs we dont want with a Python list comprehension. Alternatively I could have included it as a second argument when loading the file above if I new the names beforehand.
tabs = [x for x in tabs if x.name != "Notes"]

for tab in tabs:
    print(tab.name)

# i think this should give "Weekly pay - Gross 2018" from the file name

All
Male
Female
Full-Time
Part-Time
Male Full-Time
Male Part-Time
Female Full-Time
Female Part-Time
Weekly pay - Gross


In [3]:
# Get time and statisitcs variables from file name before looping through the tabs
# i think this should give "Weekly pay - Gross 2018" from the file name
Statistics = inputfile.split("   ",1)[1].split(".",1)[0][0:-5]
time = inputfile.split(".",3)[2][-4:]

for tab in tabs:

    # define a selection of cells as the observations
    # bear in mind this will include observations that have been aggregated to region level
    obs = tab.excel_ref('H6:Q6').expand(DOWN).is_not_blank().is_not_whitespace()
    
    # define other selections of cells to be our dimensions
    geography = tab.excel_ref('B6').expand(DOWN).is_not_blank().is_not_whitespace()
    percentiles = tab.excel_ref('H5:Q5')

    # Get dimension values from tab names:
    
    # determining gender from tab name
    if "Male" in tab.name: 
        gender = "Male"
    elif "Female" in tab.name:
        gender = "Female"
    else:
        gender = "All"
    
    # determining working pattern
    if "Full-Time" in tab.name:
        working_pattern = "Full-Time"
    elif "Part-Time" in tab.name:
        working_pattern = "Part-Time"
    else:
        working_pattern = "All"


    dimensions = [ 
        HDim(geography, "geography", DIRECTLY, LEFT), 
        HDim(percentiles, "percentiles", DIRECTLY, ABOVE), 
        # anything where the value is dervied from the tab name is going to be a constant
        HDimConst("Gender",gender), 
        HDimConst("Working Pattern", working_pattern),
        HDimConst("Time",time), 
        HDimConst("Statistics", Statistics) 
    ]


    conversionsegment = ConversionSegment(tab, dimensions, obs)
    tidy_sheet = conversionsegment.topandas()
    tidy_sheets.append(tidy_sheet)

In [4]:
savepreviewhtml(conversionsegment,previewfile)

# TODO need to replace the x's in observations with blanks/None. Wouldn't be able to add two separate datatypes into the same column in a database or cube.
# noticed that there's hierachy of areas e.g. Devon is split up into East Deven etc. 


tablepart 'Female Part-Time' written #injblock1001
javascript calculated


/workspaces/mock-transformations/pay_results.html

In [5]:
tidy_sheets[1]

# what's this DATAMARKER about in the results in the bottom cell?... allows us to record quality of data so not always empty
# TODO change data types and order of columns to have observations last and the dimensions in descending hierarchical order. 

,OBS,DATAMARKER,geography,percentiles,Gender,Working Pattern,Time,Statistics
0,224.2,NaN,K02000001,10.0,Male,All,2018,Weekly pay - Gross
1,342.7,NaN,K02000001,20.0,Male,All,2018,Weekly pay - Gross
2,376.3,NaN,K02000001,25.0,Male,All,2018,Weekly pay - Gross
3,410.6,NaN,K02000001,30.0,Male,All,2018,Weekly pay - Gross
4,479.1,NaN,K02000001,40.0,Male,All,2018,Weekly pay - Gross
...,...,...,...,...,...,...,...,...
4295,556.0,NaN,N92000002,60.0,Male,All,2018,Weekly pay - Gross
4296,639.3,NaN,N92000002,70.0,Male,All,2018,Weekly pay - Gross
4297,694.8,NaN,N92000002,75.0,Male,All,2018,Weekly pay - Gross
4298,762.3,NaN,N92000002,80.0,Male,All,2018,Weekly pay - Gross
